In [1]:
import requests
import datetime
from datetime import datetime as dt
import patoolib
import os
import pandas as pd
import sqlalchemy 
import psycopg2
from sqlalchemy import create_engine
import numpy as np
from datetime import timedelta

In [2]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])

In [3]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [4]:
now = dt.now()
delta=0
status=400
while status!=200:
    date=now- timedelta(days=delta)
    print('testing date:'+str(date))
    dia=date.strftime("%d")
    mes=date.strftime("%m")
    year=str(date.year)
    url='http://deis.minsal.cl/wp-content/uploads/'+year+'/'+mes+'/DEFUNCIONES_FUENTE_DEIS_2016_'+year+'_'+dia+mes+year+'.rar'
    print('url: '+url)
    req = requests.get(url)
    status=req.status_code
    print(status)
    if status==200:
        print('success')
    else:
        print('failure, trying previous date')
        delta+=1

testing date:2020-10-11 00:59:35.681864
url: http://deis.minsal.cl/wp-content/uploads/2020/10/DEFUNCIONES_FUENTE_DEIS_2016_2020_11102020.rar
404
failure, trying previous date
testing date:2020-10-10 00:59:35.681864
url: http://deis.minsal.cl/wp-content/uploads/2020/10/DEFUNCIONES_FUENTE_DEIS_2016_2020_10102020.rar
404
failure, trying previous date
testing date:2020-10-09 00:59:35.681864
url: http://deis.minsal.cl/wp-content/uploads/2020/10/DEFUNCIONES_FUENTE_DEIS_2016_2020_09102020.rar
404
failure, trying previous date
testing date:2020-10-08 00:59:35.681864
url: http://deis.minsal.cl/wp-content/uploads/2020/10/DEFUNCIONES_FUENTE_DEIS_2016_2020_08102020.rar
200
success


In [5]:
path='/data/ETLcache/'
now = dt.now()
timestamp = now.strftime("_%d%m%Y_%H%M%S")

In [6]:
open(path+'DEIS.rar', 'wb').write(req.content)

4105656

In [7]:
if not os.path.exists(path+'DEIS'+timestamp):
    os.makedirs(path+'DEIS'+timestamp)
    patoolib.extract_archive(path+'DEIS.rar', outdir=path+'DEIS'+timestamp)

patool: Extracting /data/ETLcache/DEIS.rar ...
patool: running /usr/bin/rar x -- /data/ETLcache/DEIS.rar
patool:     with cwd='/data/ETLcache/DEIS_11102020_005943'
patool: ... /data/ETLcache/DEIS.rar extracted to `/data/ETLcache/DEIS_11102020_005943'.


In [21]:
for f in os.listdir(path+'DEIS'+timestamp):
    if '.csv' in f:
        deis=pd.read_csv(path+'DEIS'+timestamp+'/'+f,sep=';',encoding='latin1',header=0)
    if '.xlsx' in f:
        headers=pd.read_excel(path+'DEIS'+timestamp+'/'+f,header=3,index_col=1)

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (16,17,18,19,20,21,22,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [22]:
headers=headers['Nombre de la variable']

In [23]:
headers=headers[1:]
deis.columns=headers

In [24]:
deis['Metropolitana']=0
deis.loc[deis['GLOSA_REG_RES']=='Metropolitana de Santiago','Metropolitana']=1
deis['Metropolitana']=deis['Metropolitana'].replace({0:'Resto de Chile',1:'Región Metropolitana'})

In [25]:
deis['FECHA_DEF']=pd.to_datetime(deis['FECHA_DEF'],format='%d-%m-%Y')
deis['FECHA_DEF']=deis['FECHA_DEF'].dt.to_pydatetime()

In [26]:
deis['FECHA_STR']=deis['FECHA_DEF'].dt.strftime('%d-%m-%Y')

In [27]:
deis['COVID']=deis['DIAG1'].replace({'U071':'Covid PCR+','U072':'Covid Sospechoso'})
deis.loc[~deis['DIAG1'].isin(['U071','U072']),'COVID']=np.nan

In [28]:
deis['COVID'].value_counts()

Covid PCR+          13272
Covid Sospechoso     4684
Name: COVID, dtype: int64

In [29]:
if not 'ANO_DEF' in deis.columns:
    print('creando columna: ANO_DEF')
    deis['ANO_DEF']=deis['FECHA_DEF'].dt.year

creando columna: ANO_DEF


In [31]:
for k in ['DEIS']:
    if not cnx.dialect.has_schema(cnx, k):
        print('schema '+k+' does not exist, creating it')
        cnx.execute(sqlalchemy.schema.CreateSchema(k))
    else:
        print('schema '+k+' exists, will not be created')

schema DEIS exists, will not be created


In [32]:
edad_fecha=deis.loc[(deis['ANO_DEF']==2020) & (deis['COVID'].notnull())]
edad_fecha=edad_fecha.groupby(['FECHA_DEF','EDAD_CANT'])['COVID'].count().reset_index()
edad_fecha=edad_fecha.set_index('FECHA_DEF').groupby('EDAD_CANT').rolling('7d',min_periods=1)['COVID'].mean()
edad_fecha=edad_fecha.reset_index()
edad_fecha.to_sql('DEIS_EDAD_FECHA_ROLLING', schema='DEIS',con=cnx,if_exists='replace')

In [34]:
schema='DEIS'
for y in deis['ANO_DEF'].unique():
    name='DEIS_'+str(y).split('.')[0]
    df=deis[deis['ANO_DEF']==y]
    print("creating table "+name+' ,schema: '+schema)
    df.to_sql(name, schema=schema,con=cnx,if_exists='replace')
    print("saving table"+path+name+timestamp+'.csv in cache')
    df.to_csv(path+name+timestamp+'.csv',encoding='utf-8')

creating table DEIS_2020 ,schema: DEIS
saving table/data/ETLcache/DEIS_2020_11102020_005943.csv in cache
creating table DEIS_2018 ,schema: DEIS
saving table/data/ETLcache/DEIS_2018_11102020_005943.csv in cache
creating table DEIS_2019 ,schema: DEIS
saving table/data/ETLcache/DEIS_2019_11102020_005943.csv in cache
creating table DEIS_2016 ,schema: DEIS
saving table/data/ETLcache/DEIS_2016_11102020_005943.csv in cache
creating table DEIS_2017 ,schema: DEIS
saving table/data/ETLcache/DEIS_2017_11102020_005943.csv in cache


In [41]:
os.system('jupyter nbconvert --output /home/jovyan/work/ETLdocs/' + 'ETL_covid-chile.html' + ' --to html ' + 
          '/home/jovyan/work/ETL/covid-chile.ipynb')

0